In [1]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
# import h5py
np.random.seed(1337) # for reproducibility

from keras.preprocessing import sequence
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution1D,Convolution2D, MaxPooling1D, AtrousConvolution2D
from keras.regularizers import l2, activity_l1
from keras.constraints import maxnorm
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import ModelCheckpoint, EarlyStopping

from keras.optimizers import Adam, SGD

from keras import initializations

# In[14]:

def my_init(shape, name=None):
    return initializations.normal(shape, scale=0.001, name=name)



/Users/jx/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
Using Theano backend.


In [2]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
# import h5py
np.random.seed(1337) # for reproducibility

from keras.preprocessing import sequence
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution1D,Convolution2D, MaxPooling1D
from keras.regularizers import l2, activity_l1
from keras.constraints import maxnorm
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import ModelCheckpoint, EarlyStopping

from keras.optimizers import Adam, SGD

from keras import initializations

# In[14]:

def my_init(shape, name=None):
    return initializations.normal(shape, scale=0.001, name=name)


#Load data directly from file
f = open('goodmus.txt', 'r')
y = []
for line in f:
	y.append(float(line))

f = open('goodseqs.txt', 'r')
seqs = []
for line in f:
	seqs.append(line.strip('\n'))

# ATGC to one hot encoding
seq_map = {'A': np.array([1,0,0,0]), 'T': np.array([0,1,0,0]), 'G': np.array([0,0,1,0]), 'C': np.array([0,0,0,1])}

#sequences to matrix form
X = []
for seq in seqs:
	X.append(np.vstack([seq_map[s] for s in seq]))

# zero pad sequences (end)
max_len = 0
for s in X:
	if len(s)>max_len:
		max_len = len(s)
for i in range(len(X)):
	p = max_len - X[i].shape[0]
	if p>0:
		X[i] = np.vstack([X[i], np.zeros((p,4))])

X= np.asarray(X)
y = np.asarray(y)
print X.shape


(35063, 113, 4)


In [3]:
#shuffle indices...not data!
idx = range(len(X))
np.random.shuffle(idx)
train_idx = idx[:int(len(X)*.8)]
valid_idx = idx[int(len(X)*.8):int(len(X)*.9)]

#split into train/valid
X_train = X[train_idx]
X_valid = X[int(len(X)*.8):int(len(X)*.9)]
X_test = X[int(len(X)*.9):]

y_train = y[:int(len(X)*.8)]
y_valid = y[int(len(X)*.8):int(len(X)*.9)]
y_test = y[int(len(X)*.9):]

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[2], 1, X_train.shape[1])) #(N, F, H, W)
X_valid = np.reshape(X_valid, (X_valid.shape[0], X_valid.shape[2], 1, X_valid.shape[1])) #(N, F, H, W)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[2], 1, X_test.shape[1])) #(N, F, H, W)

print X_train.shape


(28050, 4, 1, 113)


In [9]:
lr = 1e-4#learning rate
reg = 1e-6

print 'building model'
model = Sequential()
model.add(AtrousConvolution2D(128, 4, 4,atrous_rate=(2,2) border_mode='same', input_shape=(X_train.shape[1], 1, X_train.shape[3]), W_regularizer=l2(reg), init=my_init))
model.add(Activation('tanh'))
# model.add(AtrousConvolution2D(256, 4, 4,atrous_rate=(2,2) border_mode='same', W_regularizer=l2(reg), init=my_init))
# model.add(Activation('tanh'))
# model.add(AtrousConvolution2D(512, 4, 4,atrous_rate=(2,2) border_mode='same', W_regularizer=l2(reg), init=my_init))
# model.add(Activation('tanh'))
# model.add(Flatten())

# model.add(Dense(1000))
# model.add(Dense(1))
# model.add(Activation('tanh')) #[-1, 1]


# In[17]:
adam = Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

# sgd = SGD(lr=lr, momentum=0.9)
model.compile(loss='mse',
              optimizer='adam', metrics=['mse'])


print 'compiling model'

SyntaxError: invalid syntax (<ipython-input-9-30718f26b718>, line 6)

earlystopper = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
model.fit(X_train, y_train,nb_epoch=3, batch_size=512, verbose=1, show_accuracy=True, validation_data=(X_valid, y_valid),callbacks=[earlystopper])



In [20]:
%matplotlib 

out=model.predict(X_test, batch_size=512, verbose=1)
plt.plot(y_test,out,'ro')

model.summary()


Using matplotlib backend: MacOSX
3507/3507 [==============================] - 22s    
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_7 (Convolution2D)  (None, 128, 1, 113)   8320        convolution2d_input_3[0][0]      
____________________________________________________________________________________________________
activation_9 (Activation)        (None, 128, 1, 113)   0           convolution2d_7[0][0]            
____________________________________________________________________________________________________
convolution2d_8 (Convolution2D)  (None, 256, 1, 113)   524544      activation_9[0][0]               
____________________________________________________________________________________________________
activation_10 (Activation)       (None, 256, 1, 113)   0           convolution2d_8[0][0]            
_____